In [76]:
# Import packages required
import pandas as pd

In [77]:
# Import messy_data from Excel
messy_data = pd.read_excel('messy_order_data.xlsx')

# Output head of mess_data to the screen
messy_data.head()

,Order Number,Customer,Units Ordered,Price Per Unit (£s),Order Email,Client Address
0,ORDER-06,CUSTOMER-1,100,£---289,DANA.NGUYEN@CUSTOMER-1.COM,PSC 4115\nBox 7815\nAPO\nENGLAND\nAA 41945
1,ORDER-11,CUSTOMER-1,138,£---896,DANA.NGUYEN@CUSTOMER-1.COM,PSC 4115\nBox 7815\nAPO\nENGLAND\nAA 41945
2,ORDER-12,CUSTOMER-1,136,£---905,DANA.NGUYEN@CUSTOMER-1.COM,PSC 4115\nBox 7815\nAPO\nENGLAND\nAA 41945
3,ORDER-15,CUSTOMER-1,79,£---325,CHERYL.BRADLEY@CUSTOMER-1.COM,PSC 4115\nBox 7815\nAPO\nENGLAND\nAA 41945
4,ORDER-16,CUSTOMER-1,132,"£---1,058",DANA.NGUYEN@CUSTOMER-1.COM,PSC 4115\nBox 7815\nAPO\nENGLAND\nAA 41945


## Step 1: Cleaning Price Per Unit Value

See last article for details regarding this.  This is now implemented in a more pythonic way using REGEX, which will be covered in Article 4.

In [78]:
# Implement cleaning activity from previous article
messy_data['Price Per Unit (£s)'] = messy_data['Price Per Unit (£s)'].str.replace('£---|,','', regex=True).astype(float)

## Step 2: Calculating the Total Order Value

Pandas provides us with a simple way to apply formulars to data series that have the same shape (in Excel speak, columns of data that have the same number of rows populated).  We can simply apply the mathematical operations we want to directly to the columns. As the number of elements in each series is the same, Pandas knows to apply the function to corresponding values of the same index.

This means we can calculate the total order value per line by implementing the following code

In [79]:
# Calculate Total Order Value
messy_data['Total Order Value (£s)'] = messy_data['Units Ordered'] *  messy_data['Price Per Unit (£s)']

# Output head of mess_data to the screen (limited to the variables that have been used in calcualtion)
messy_data[['Units Ordered','Price Per Unit (£s)','Total Order Value (£s)']].head()

,Units Ordered,Price Per Unit (£s),Total Order Value (£s)
0,100,289.0,28900.0
1,138,896.0,123648.0
2,136,905.0,123080.0
3,79,325.0,25675.0
4,132,1058.0,139656.0


## Step 3: Splitting Client Address Column into Multiple Variables

Noting that the data is in the same format for all values, (Street / Box Number / Country / Mail Code), the following code will split the data and return individual variables for each element.

As we have duplicated the data from the Client Address in the new individual variables, we can now drop the original data from the messy_data dataframe.  The last line of code does this for us and the variable ```axis = 1```, tells the ```drop()``` function we are looking to drop a column.

In [80]:
# Define a list of new variable names for the seperated data from the Client Address
address_details = ['Street','Mail Box','County','Country','Mail Code']

# Split Client Data in to seperate columns and assign to the columns named in the list address_details in the messy_data dataframe
messy_data[address_details] = messy_data['Client Address'].str.split('\n',expand=True)

# Remove the original Client Address variable from the data
messy_data = messy_data.drop(['Client Address'],axis=1)

# Output head of mess_data to the screen
messy_data.head()

,Order Number,Customer,Units Ordered,Price Per Unit (£s),Order Email,Total Order Value (£s),Street,Mail Box,County,Country,Mail Code
0,ORDER-06,CUSTOMER-1,100,289.0,DANA.NGUYEN@CUSTOMER-1.COM,28900.0,PSC 4115,Box 7815,APO,ENGLAND,AA 41945
1,ORDER-11,CUSTOMER-1,138,896.0,DANA.NGUYEN@CUSTOMER-1.COM,123648.0,PSC 4115,Box 7815,APO,ENGLAND,AA 41945
2,ORDER-12,CUSTOMER-1,136,905.0,DANA.NGUYEN@CUSTOMER-1.COM,123080.0,PSC 4115,Box 7815,APO,ENGLAND,AA 41945
3,ORDER-15,CUSTOMER-1,79,325.0,CHERYL.BRADLEY@CUSTOMER-1.COM,25675.0,PSC 4115,Box 7815,APO,ENGLAND,AA 41945
4,ORDER-16,CUSTOMER-1,132,1058.0,DANA.NGUYEN@CUSTOMER-1.COM,139656.0,PSC 4115,Box 7815,APO,ENGLAND,AA 41945


## Step 4: Extract the Name of The Ordering Individual

Using ‘@’ as a delimiter, we can adapt the technique applied in step 3 to extract the first and surnames from the email address. 

Before we carry out the data cleansing activity, lets explore the output the function when '@' is used as a delimiter

In [81]:
# Apply str.split to Order Email, using '@' as a delimiter (return head only)
messy_data['Order Email'].str.split('@',expand=True).head()

,0,1
0,DANA.NGUYEN,CUSTOMER-1.COM
1,DANA.NGUYEN,CUSTOMER-1.COM
2,DANA.NGUYEN,CUSTOMER-1.COM
3,CHERYL.BRADLEY,CUSTOMER-1.COM
4,DANA.NGUYEN,CUSTOMER-1.COM


We can see that this has returned two columns of data.  For our analysis, we only require the first of these, the zeroth indexed column.  We can isolate this by indexing with [0]

In [82]:
# Apply str.split to Order Email, using '@' as a delimiter and return first column head only
messy_data['Order Email'].str.split('@',expand=True)[0].head()

0       DANA.NGUYEN
1       DANA.NGUYEN
2       DANA.NGUYEN
3    CHERYL.BRADLEY
4       DANA.NGUYEN
Name: 0, dtype: object

The following code implements the required cleansing step and replaces the . with a space to result in a corectly formatted name.

In [83]:
# Extract the first name and last name from the Order Email and add to messy_data
messy_data['Order Name'] = messy_data['Order Email'].str.split('@',expand=True)[0].str.replace('.',' ')

messy_data.head()

,Order Number,Customer,Units Ordered,Price Per Unit (£s),Order Email,Total Order Value (£s),Street,Mail Box,County,Country,Mail Code,Order Name
0,ORDER-06,CUSTOMER-1,100,289.0,DANA.NGUYEN@CUSTOMER-1.COM,28900.0,PSC 4115,Box 7815,APO,ENGLAND,AA 41945,DANA NGUYEN
1,ORDER-11,CUSTOMER-1,138,896.0,DANA.NGUYEN@CUSTOMER-1.COM,123648.0,PSC 4115,Box 7815,APO,ENGLAND,AA 41945,DANA NGUYEN
2,ORDER-12,CUSTOMER-1,136,905.0,DANA.NGUYEN@CUSTOMER-1.COM,123080.0,PSC 4115,Box 7815,APO,ENGLAND,AA 41945,DANA NGUYEN
3,ORDER-15,CUSTOMER-1,79,325.0,CHERYL.BRADLEY@CUSTOMER-1.COM,25675.0,PSC 4115,Box 7815,APO,ENGLAND,AA 41945,CHERYL BRADLEY
4,ORDER-16,CUSTOMER-1,132,1058.0,DANA.NGUYEN@CUSTOMER-1.COM,139656.0,PSC 4115,Box 7815,APO,ENGLAND,AA 41945,DANA NGUYEN


## Step 5: Correcting the Capitalization of Text

First, lets inspect the Order Name and Country Variables

In [84]:
messy_data[['Order Name','Country']].head()

,Order Name,Country
0,DANA NGUYEN,ENGLAND
1,DANA NGUYEN,ENGLAND
2,DANA NGUYEN,ENGLAND
3,CHERYL BRADLEY,ENGLAND
4,DANA NGUYEN,ENGLAND


We can see that there are two variables which are fully capitalized.  As they contain proper nouns, we want to capitalize the first letter of them only

In [85]:
# Update the Country variable to be a title case (i.e. capitalise the first letter of each word)
messy_data['Country'] = messy_data['Country'].str.title()

# Update the Order Name variable to be a title case
messy_data['Order Name'] = messy_data['Order Name'].str.title()

# Output the head of the messy_data data fram to the screen
messy_data.head()

,Order Number,Customer,Units Ordered,Price Per Unit (£s),Order Email,Total Order Value (£s),Street,Mail Box,County,Country,Mail Code,Order Name
0,ORDER-06,CUSTOMER-1,100,289.0,DANA.NGUYEN@CUSTOMER-1.COM,28900.0,PSC 4115,Box 7815,APO,England,AA 41945,Dana Nguyen
1,ORDER-11,CUSTOMER-1,138,896.0,DANA.NGUYEN@CUSTOMER-1.COM,123648.0,PSC 4115,Box 7815,APO,England,AA 41945,Dana Nguyen
2,ORDER-12,CUSTOMER-1,136,905.0,DANA.NGUYEN@CUSTOMER-1.COM,123080.0,PSC 4115,Box 7815,APO,England,AA 41945,Dana Nguyen
3,ORDER-15,CUSTOMER-1,79,325.0,CHERYL.BRADLEY@CUSTOMER-1.COM,25675.0,PSC 4115,Box 7815,APO,England,AA 41945,Cheryl Bradley
4,ORDER-16,CUSTOMER-1,132,1058.0,DANA.NGUYEN@CUSTOMER-1.COM,139656.0,PSC 4115,Box 7815,APO,England,AA 41945,Dana Nguyen


## Step 6: Identifying and Removing Duplicate Rows of Data

By importing and visually inspecting the compay_data dataframe, we can see that there are duplicate entries in rows 3 and 4

In [86]:
# Read in the company data to use in the look up
company_data = pd.read_excel('company_details.xlsx')

# Print duplicate data and shape of data to the screen
print('Shape of company_data dataframe:',company_data.shape)
company_data

Shape of company_data dataframe: (5, 3)


,Customer,Customer Name,Sales Rep
0,CUSTOMER-1,"Perez, Torres and Moore",Tim Brady
1,CUSTOMER-2,Alvarez Group,Joshua Dyer
2,CUSTOMER-3,Valencia-Sanchez,Carrie Brady
3,CUSTOMER-4,"Ryan, Randolph and Armstrong",Carrie Brady
4,CUSTOMER-4,"Ryan, Randolph and Armstrong",Carrie Brady


The Pandas function ```drop_duplicates()``` provides a quick way to remove these from our dataframe.  The function is quite versatile and can be applied in many ways.  

In [87]:
# Remove duplicate rows from the company_data dataframe 
company_data = company_data.drop_duplicates()

# Print deduplicated data and shape of data to the screen
print('Shape of company_data dataframe:',company_data.shape)
company_data

Shape of company_data dataframe: (4, 3)


,Customer,Customer Name,Sales Rep
0,CUSTOMER-1,"Perez, Torres and Moore",Tim Brady
1,CUSTOMER-2,Alvarez Group,Joshua Dyer
2,CUSTOMER-3,Valencia-Sanchez,Carrie Brady
3,CUSTOMER-4,"Ryan, Randolph and Armstrong",Carrie Brady


## Step 7: Looking Up Values

In [88]:
# Print the shape of the meesy_data dataframe to the screen prior to merge
print('Shape of messy_data prior to merge:', messy_data.shape)

# Merge data sets and lookup values in the messy_data and customer_data via an Inner Join on the 'Customer' variable
clean_data = messy_data.merge(company_data, on = 'Customer')

# Print the shape of the meesy_data dataframe to the screen prior to merge
print('Shape of clean_data post merge:', clean_data.shape)

# Print head of clean_data to the screen
clean_data.head()

Shape of messy_data prior to merge: (50, 12)
Shape of clean_data post merge: (50, 14)


,Order Number,Customer,Units Ordered,Price Per Unit (£s),Order Email,Total Order Value (£s),Street,Mail Box,County,Country,Mail Code,Order Name,Customer Name,Sales Rep
0,ORDER-06,CUSTOMER-1,100,289.0,DANA.NGUYEN@CUSTOMER-1.COM,28900.0,PSC 4115,Box 7815,APO,England,AA 41945,Dana Nguyen,"Perez, Torres and Moore",Tim Brady
1,ORDER-11,CUSTOMER-1,138,896.0,DANA.NGUYEN@CUSTOMER-1.COM,123648.0,PSC 4115,Box 7815,APO,England,AA 41945,Dana Nguyen,"Perez, Torres and Moore",Tim Brady
2,ORDER-12,CUSTOMER-1,136,905.0,DANA.NGUYEN@CUSTOMER-1.COM,123080.0,PSC 4115,Box 7815,APO,England,AA 41945,Dana Nguyen,"Perez, Torres and Moore",Tim Brady
3,ORDER-15,CUSTOMER-1,79,325.0,CHERYL.BRADLEY@CUSTOMER-1.COM,25675.0,PSC 4115,Box 7815,APO,England,AA 41945,Cheryl Bradley,"Perez, Torres and Moore",Tim Brady
4,ORDER-16,CUSTOMER-1,132,1058.0,DANA.NGUYEN@CUSTOMER-1.COM,139656.0,PSC 4115,Box 7815,APO,England,AA 41945,Dana Nguyen,"Perez, Torres and Moore",Tim Brady


## Export back to Excel

In [89]:
# Define order of columns for outputed data
column_ordering = ['Order Number', 'Customer', 'Customer Name','Sales Rep','Units Ordered',
                   'Price Per Unit (£s)','Total Order Value (£s)', 'Order Name', 
                   'Order Email', 'Street', 'Mail Box', 'County', 'Country', 'Mail Code']

# Export clean data to excel file for analysis and processing
clean_data[column_ordering].to_excel('Clean Order Data.xlsx',index = False)